In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
import scipy
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedKFold
from util import plot_history
import feature_builder

Cargo datasets

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Armo features

In [3]:
X_train = feature_builder.process_dataset(train_df)
y_train = train_df['target'].values

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
X_train.columns

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

model = create_model()
history = model.fit(X_train, y_train, epochs=15, shuffle=True, validation_split=0.2)
plot_history(history)

In [ ]:
test = feature_builder.process_dataset(test_df)
prediction = model.predict_classes(test)
pd.Series([x[0] for x in prediction]).value_counts()

Calculemos el test para kaggle

In [34]:
ids = test_df['id']

final_test = feature_builder.process_dataset(test_df)
prediction = model.predict_classes(final_test)
final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

,target
id,
0,0
2,1
3,1
9,1
11,1
...,...
10861,1
10865,1
10868,1


In [35]:
final_df['target'].value_counts()

0    2005
1    1258
Name: target, dtype: int64

In [36]:
final_df.count()

target    3263
dtype: int64

In [37]:
final_df.to_csv('nn.csv')